In [1]:
from crystalbuilder import *
import crystalbuilder.geometry as geo
import matplotlib.pyplot as plt
import tidy3d as td
import crystalbuilder.convert as cv
import numpy as np
import vedo

Error: Lumpy and/or the Lumerical API were not found.


Here we will create a simulation of the core-shell diamond crystal. This is a rod-connected diamond with a cubic conventional cell. We'll set it up in the usual manner, but we can use an actual distance in nanometers for our lattice constant. `a_mag` is this constant in microns (Tidy3D default units)

In [2]:
lattice_constant = 700 #nm

a1 = [1, 0, 0]
a2 = [0, 1 ,0]
a3 = [0, 0, 1]

a_mag = lattice_constant/1000 #convert to microns
geo_lattice = lattice.Lattice(a1, a2, a3, magnitude = [a_mag, a_mag, a_mag])

We'll load our points from Bilbao (or a local copy if already done before). We don't scale any of this using the lattice constant.  

In [3]:
diamond = bilbao.SpaceGroup(227)
pt = 1/8
points = diamond.calculate_points(point_list=[(pt, pt, pt)]) #This is the 8a wyckoff position

The IUCr tabulated general positions are in units of the lattice constant. After we connect the nearest neighbors, we convert to our micron units and scale the structure. This is done by passing `a_mag` to `NearestNeighbors()`. 

In [4]:
radius = .05 #microns

crystal_core = geo.NearestNeighbors(points, radius=radius, neighborhood_range=.45, a_mag=a_mag)

For our core-shell structure, we will just create a slightly larger (by `shell_radius`) set of connected rods. 

In [5]:
shell_radius = .02 #microns

crystal_shell = geo.NearestNeighbors(points, radius=(radius+shell_radius), neighborhood_range=.45, a_mag=a_mag)

The geometry is just a combination of these two lattices, **making sure to include the core *after* the shell**. In Tidy3D, items later in the list will override earlier ones.

In [6]:
geometry = [crystal_shell, crystal_core]

In [7]:
a1_reps = 6
a2_reps = 6
a3_reps = 4
crystal = geo_lattice.tile_geogeometry(geometry, a1_reps, a2_reps, a3_reps )
print(len(crystal[0]))
print(len(crystal[1]))

2304
2304


In [8]:
scene = viewer.visualize(crystal[1], c='white', res=5)
viewer.add_to_visualizer(crystal[0], scene, c='red', alpha=.5, res=5)


# scene.show().close()

In [9]:
shell = cv.geo_to_tidy3d(crystal[0], 3.5, name="Shell", material_name="HPCVD-Si")
core = cv.geo_to_tidy3d(crystal[1], 1.9, name="Core", material_name = "FEBID-Pt")

In [14]:
# create source
lda0 = 1.25  # wavelength of interest (length scales are micrometers in Tidy3D)
src_wavelengths = [.7, 1.6]

src_freqs = np.sort(td.C_0 / np.asarray(src_wavelengths))
freq0 = td.C_0 / np.mean(src_wavelengths)  # frequency of interest
print(src_freqs)


[1.87370286e+14 4.28274940e+14]


In [17]:

def make_sources(frequencies, size, theta_range=[0, 1.2], phi_range = [0, 2*np.pi], number_of_angles = 5, **kwargs):
    time_profile = td.GaussianPulse.from_frequency_range(fmin=frequencies[0], fmax=frequencies[1])
    center = kwargs.get("center", (0,0,2))
    direction = kwargs.get("direction", '-')
    phi = np.arange(phi_range[0], phi_range[1], .275)
    theta = np.linspace(theta_range[0], theta_range[1], number_of_angles)
    source_list = []
    for aang in theta:
        if aang == 0:
            scaled_time_profile = td.GaussianPulse.from_frequency_range(fmin=frequencies[0], fmax=frequencies[1], amplitude=len(phi))
            source = td.GaussianBeam(
                size = size,
                center = center,
                source_time = scaled_time_profile,
                direction=direction,
                waist_radius=2.5,
                angle_theta=aang,
                angle_phi=0,
                name="Normal Incidence"
            )
            source_list.append(source)
            pass
        else:
            for rang in phi:
                source = td.GaussianBeam(
                    size = size,
                    center = center,
                    source_time = time_profile,
                    direction=direction,
                    waist_radius=2.5,
                    angle_theta=aang,
                    angle_phi=rang
                )
                source_list.append(source)
    
    return source_list

sources = make_sources(src_freqs, theta_range=[0, 1], size=[3,3,0], number_of_angles = 15)


nmonitor_freqs = 25 #Number of frequencies to sample at.

monitor_freqs = np.linspace(src_freqs[0], src_freqs[1] ,nmonitor_freqs)

farfield_monitor = td.FieldProjectionAngleMonitor(center=[0,0,2.25], 
                                                  size=[td.inf,td.inf,0],
                                                  freqs=(monitor_freqs),
                                                  name='farfield_monitor',
                                                  phi = np.linspace(0, 2*np.pi, 100),
                                                  theta=np.linspace(0,1.2, 70))

flux_t_monitor = td.FieldMonitor(center=[0,0,-2], 
                                size=[td.inf,td.inf,0],
                                freqs=(monitor_freqs),
                                name='trans_monitor'  
)

flux_r_monitor = td.FieldMonitor(center=[0,0,2.25], 
                                size=[td.inf,td.inf,0],
                                freqs=(monitor_freqs),
                                name='refl_monitor')

monitor_list = [farfield_monitor, flux_t_monitor, flux_r_monitor]
sim = td.Simulation(
    size=(10, 10, 6),  # simulation domain size
    center = (0,0,0),
    grid_spec=td.GridSpec.auto(
        min_steps_per_wvl=20
    ),  # automatic nonuniform FDTD grid with 25 grids per wavelength in the material
    structures=[shell, core],
    sources=sources,
    monitors=monitor_list,
    run_time=3e-13,  # physical simulation time in second
) # type: ignore
sim.plot_3d()





In [18]:
from tidy3d import web

task_id = web.upload(sim, task_name=f"{a1_reps}x{a2_reps}x{a3_reps}-CoreShell-ManyGaussianswMonitors")

07:31:01 Central Daylight Time Created task                                     
                               '6x6x4-CoreShell-ManyGaussianswMonitors' with    
                               task_id                                          
                               'fdve-6d4016a4-f6f6-4a32-9039-6299c3ce6722' and  
                               task_type 'FDTD'.

                               View task using web UI at                        
                               ]8;id=411566;https://tidy3d.simulation.cloud/workbench?taskId=fdve-6d4016a4-f6f6-4a32-9039-6299c3ce6722\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=383036;https://tidy3d.simulation.cloud/workbench?taskId=fdve-6d4016a4-f6f6-4a32-9039-6299c3ce6722\taskId]8;;\
                               ]8;id=411566;https://tidy3d.simulation.cloud/workbench?taskId=fdve-6d4016a4-f6f6-4a32-9039-6299c3ce6722\=]8;;\]8;id=342685;https://tidy3d.simulation.cloud/workbench?taskId=fdve-6d4016a4-f6f6-4a32-9039-6299c3ce6722\fdve]8;;\]8;id=411566;https://tidy3d.simulation.cloud/workbench?taskId=fdve-6d4016a4-f6f6-4a32-9039-6299c3ce6722\-6d4016a4-f6f6-4a32-9039-6299c3ce6722']8;;\.

                               Task folder: ]8;id=559934;https://tidy3d.simulation.cloud/folders/767b270d-f545-4c6b-9509-691c0e60105a\'default']8;;\.

c:\Users\Brandon\miniconda3\envs\simulation\Lib\site-packages\rich\live.py:231: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

07:32:09 Central Daylight Time Maximum FlexCredit cost: 0.211. Minimum cost     
                               depends on task execution details. Use           
                               'web.real_cost(task_id)' to get the billed       
                               FlexCredit cost after a simulation run.